In [2]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel, RandomForest
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time


# Read the dataset that was previously splitted due to jupyter's maximum file size issue
rawData = sc.textFile("*.csv")
def prepare(line):
    values = line.split(',')
    map(float, values)
    featureVector = Vectors.dense(values[:-1])
    # Decision tree labels varies from 0 to n-1
    label = float(values[-1])-1
    return LabeledPoint(label, featureVector)

def new_prepare(line):
    values = line.split(',')
    map(float, values)
    wilderness = values[10:14].index('1')
    soil = values[14:54].index('1')
    featureVector = Vectors.dense(values[0:10]+[wilderness,soil])
    # Decision tree labels varies from 0 to n-1
    label = float(values[-1])-1
    return LabeledPoint(label, featureVector)

data = rawData.map(lambda line : new_prepare(line))

trainData, cvData, testData = data.randomSplit([0.8, 0.1, 0.1])

trainData.cache()
cvData.cache()
testData.cache()

PythonRDD[9] at RDD at PythonRDD.scala:48

In [ ]:
# Treino: trainData
# Teste: cvData
# Descobrir a configuração dos hyperparameters

start_time = time.time()


model = DecisionTree.trainClassifier(trainData, numClasses=7, categoricalFeaturesInfo={10: 4, 11: 40},
    impurity='entropy', maxDepth=20, maxBins=300)
predictions = model.predict(cvData.map(lambda x: x.features))
labelsAndPredictions = cvData.map(lambda lp: lp.label).zip(predictions) 
m = MulticlassMetrics(labelsAndPredictions)

print m.precision()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Treino: trainData + cvData
# Teste: testData
# Verificar a precisão da Decision Tree usando o conjunto de teste
start_time = time.time()


model = DecisionTree.trainClassifier(trainData.union(cvData), numClasses=7, categoricalFeaturesInfo={}, 
                                     impurity='entropy', maxDepth=20, maxBins=300)
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions) 
m = MulticlassMetrics(labelsAndPredictions)

print m.precision()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Treino: trainData + cvData
# Teste: testData
# Random Forest com 20 Decision Trees

start_time = time.time()


forest = RandomForest.trainClassifier(trainData.union(cvData), numClasses=7, categoricalFeaturesInfo={10: 4, 11: 40}, numTrees=20,
                                      featureSubsetStrategy='auto', impurity='entropy', maxDepth=30, maxBins=300)
predictions = forest.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions) 
k = MulticlassMetrics(labelsAndPredictions)
print k.precision()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
layers = [11,12,11,7]
# trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# model = trainer.fit(trainData)

# result = model.transform(test)
# xy = result.select("prediction", "label")
# evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
# print ("Precision: "+str(evaluator.evaluate(xy)))